In [3]:
pip install django requests googlemaps yelpapi openai scikit-learn numpy pandas

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
   --- ------------------------------------ 0.8/8.3 MB 5.6 MB/s eta 0:00:02
   ------ --------------------------------- 1.3/8.3 MB 4.5 MB/s eta 0:00:02
   -------- ------------------------------- 1.8/8.3 MB 3.5 MB/s eta 0:00:02
   --------------- ------------------------ 3.1/8.3 MB 4.1 MB/s eta 0:00:02
   ----------------- ---------------------- 3.7/8.3 MB 3.8 MB/s eta 0:00:02
   ------------------------- -------------- 5.2/8.3 MB 4.5 MB/s eta 0:00:01
   ------------------------------- -------- 6.6/8.3 MB 4.7 MB/s eta 0:00:01
   ---------------------------------------  8.1/8.3 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------- 8.3/8.3 MB 5.1 MB/s eta 0:00:00
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40747 sha256=4e2ecb67243806fb223b209dc43290fdac2978833624573220e

In [5]:
from googlemaps import Client as GoogleMaps
from django.conf import settings

def get_restaurant_data(village_place_id, latitude, longitude):
    gmaps = GoogleMaps(key=settings.API_KEYS['GOOGLE_MAPS'])
    
    # Fetch details for the Village restaurant
    village_details = gmaps.place(
        place_id=village_place_id,
        fields=["name", "formatted_address", "opening_hours", "rating", "price_level"]
    )["result"]
    
    # Search for top-rated restaurants nearby
    nearby_restaurants = gmaps.places_nearby(
        location=(latitude, longitude),
        radius=2000,
        type="restaurant",
        rank_by="prominence"
    )["results"]
    
    # Filter top 5 based on ratings
    top_restaurants = sorted(
        nearby_restaurants, key=lambda x: x.get("rating", 0), reverse=True
    )[:5]
    
    return village_details, top_restaurants


In [7]:
def get_busy_times(village_place_id):
    gmaps = GoogleMaps(key=settings.API_KEYS['GOOGLE_MAPS'])
    place_details = gmaps.place(
        place_id=village_place_id,
        fields=["current_popularity"]
    )
    return place_details.get("result", {}).get("current_popularity", "N/A")


In [9]:
import requests

def get_weather_data(lat, lon):
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={settings.API_KEYS['OPENWEATHER']}"
    response = requests.get(url).json()
    temp_kelvin = response["main"]["temp"]
    temp_fahrenheit = (temp_kelvin - 273.15) * 9/5 + 32
    weather_condition = response["weather"][0]["main"]
    return temp_fahrenheit, weather_condition


In [15]:
from django.shortcuts import render

def pricing_view(request):
    village_place_id = "ChIJyxdI5Y8j_4kR9em1cFjY2AQ"  # Replace with actual Place ID
    village_lat = 40.768  # Replace with actual latitude
    village_lon = -73.525  # Replace with actual longitude
    
    # Fetch restaurant data
    village_details, top_restaurants = get_restaurant_data(
        village_place_id, village_lat, village_lon
    )
    
    # Fetch busy times
    busy_times = get_busy_times(village_place_id)
    
    # Fetch weather data
    temp_fahrenheit, weather_condition = get_weather_data(
        lat=village_lat, lon=village_lon
    )
    
    # Predict menu prices
    menu_items = ["Paneer Tikka", "Butter Chicken", "Naan"]  # Example menu items
    predicted_prices = predict_prices(
        menu_items=menu_items,
        temp_fahrenheit=temp_fahrenheit,
        weather_condition=weather_condition,
        busy_times=busy_times or 50  # Default busy times to average
    )
    
    # Prepare context for rendering
    context = {
        "village": village_details,
        "similar_restaurants": top_restaurants,
        "busy_times": busy_times,
        "weather": {"temp": temp_fahrenheit, "condition": weather_condition},
        "predicted_prices": predicted_prices,
    }
    #return render(request, "pricing.html", context)
    return render(request, "pricing.html", context)
